In [4]:
from transform_ps.etl.util import clear_cwd_cache

### Imports

In [1]:
import datetime as dt

import seaborn as sns


In [2]:
from transform_ps.etl.data_type import DataFactory

### Setup

In [3]:
factory = DataFactory()

In [4]:
tix = factory.create_data_type('ticket')

### Get Data

In [5]:
tix.get_data(fys=[17, 18, 19])

reading from cache
Finished prepare_data in 1.5976 secs


### Filter

In [45]:
pops = tix.type.filters['series'](tix.working, ['Pops']).reset_index(drop=True)

In [46]:
pops = tix.type.filters['paid'](pops, 'paid_amt').reset_index(drop=True)

In [47]:
pops = tix.type.transform['price_type_group'](pops)

### Perf Date Mapper

In [50]:
perfs = list(pops['perf_dt'].drop_duplicates())

In [51]:
mapper = {}
for perf in perfs:
    if perf - dt.timedelta(1) in perfs:
        mapper.update({perf: perf - dt.timedelta(1)})
    else:
        mapper.update({perf: perf})

In [52]:
pops['perf_dt'] = pops['perf_dt'].map(mapper)

In [53]:
concert_numbers = {dt: i % 7 + 1 for i, dt in enumerate(pops['perf_dt'].drop_duplicates())}

In [54]:
pops['concert_number'] = pops['perf_dt'].map(concert_numbers)

### Format Data

In [55]:
pops = pops[['concert_number', 'dow', 'attended', 'fy', 'paid_amt', 'price_type_group']].reset_index(drop=True)

In [56]:
pops['attended'] = pops['attended'].map(lambda x: 1 if x == 'Attended' else 0)

In [60]:
grouped = pops.groupby(['fy', 'concert_number', 'dow', 'price_type_group']).agg({
    'paid_amt': ['sum', 'count'],
    'attended': 'sum',
}).reset_index()

In [62]:
grouped.columns = ['fy', 'concert_number', 'dow', 'price_type_group', 'revenue', 'units', 'attended']

In [64]:
grouped.to_excel('out.xlsx')